In [1]:
import os
os.chdir('../')

In [2]:
import data
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import preprocess_utils.session2vec as sess2vec

tqdm.pandas()

Using TensorFlow backend.


In [3]:
train = data.train_df('small')

In [4]:
train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
1,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3
2,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
3,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
4,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1


In [35]:
len(train)

400331

In [5]:
def count_sessions_of_class_1(df):
    def last_click(g):
        clickouts = g[g.action_type == 'clickout item']
        if len(clickouts) > 0:
            return clickouts.iloc[-1].ref_class
    
    return df[['user_id','session_id','action_type','ref_class']].groupby(['user_id','session_id']).progress_apply(last_click).describe()

In [6]:
labelled_df = sess2vec.add_reference_binary_labels(train, 'action_type', 'clickout item', only_clickouts=True)

100%|██████████| 400331/400331 [00:08<00:00, 44700.12it/s]


Before resampling:

In [7]:
count_sessions_of_class_1(labelled_df)

100%|██████████| 69412/69412 [00:57<00:00, 1210.07it/s]


count    63129.000000
mean         0.376673
std          0.484556
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
dtype: float64

In [8]:
def ref_class_is_1(g):
    clickout = g[g.action_type == 'clickout item']
    return len(clickout) > 0 and clickout.iloc[-1]['ref_class'] == 1

In [43]:
def resample_sessions(df, by, when):
    """ Resample sessions that satisfy the specified condition """
    assert 1 <= by <= 2
    by -= 1
    orig_len = len(df)
    print('Resampling sessions, ETA: {:.2f} minutes...'.format(orig_len * 0.0000025))
    temp_df = df.groupby(['user_id','session_id']).filter(lambda g: when(g) and np.random.rand() < by)
    temp_df.loc[:,'user_id'] = temp_df['user_id'] + '***'
    temp_df.loc[:,'session_id'] = temp_df['session_id'] + '***'
    
    # create new df
    new_len = orig_len + len(temp_df)
    new_idx = -np.ones(new_len, dtype='int')
    new_idx[0:orig_len] = df.index.values
    res_df = pd.DataFrame(columns=df.columns, index=new_idx)
    res_df.iloc[0:orig_len,:] = df.values
    res_df.iloc[orig_len:,:] = temp_df.values
    return res_df.astype(df.dtypes)

In [45]:
%time resampled_df = resample_sessions(labelled_df, by=1.5, when=ref_class_is_1)

Resampling sessions, ETA: 1.00 minutes...


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


CPU times: user 1min 20s, sys: 868 ms, total: 1min 21s
Wall time: 1min 21s


In [39]:
resampled_df.tail()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,ref_class,$random
-1,S4D78FQEWU68***,85029d7d3ec94***,1541070466,12,clickout item,320756,PH,"Tagaytay City, Philippines",desktop,NaN,320756|2039291|1327558|5493632|3152076|3153080...,68|184|112|55|72|138|56|66|48|26|131|76|30|38|...,1,1,0.883079
-1,S4D78FQEWU68***,85029d7d3ec94***,1541070490,13,clickout item,320756,PH,"Tagaytay City, Philippines",desktop,NaN,320756|2039291|1327558|5493632|3152076|3153080...,68|184|112|55|72|138|56|66|48|26|131|76|30|38|...,1,1,0.055897
-1,S4D78FQEWU68***,85029d7d3ec94***,1541070548,23,interaction item image,320756,PH,"Tagaytay City, Philippines",desktop,NaN,NaN,NaN,10,0,0.514032
-1,S4D78FQEWU68***,85029d7d3ec94***,1541070548,24,interaction item image,521651,PH,"Tagaytay City, Philippines",desktop,NaN,NaN,NaN,1,0,0.921567
-1,S4D78FQEWU68***,85029d7d3ec94***,1541070548,25,search for destination,"Tagaytay City, Philippines",PH,"Tagaytay City, Philippines",desktop,NaN,NaN,NaN,1,0,0.706871


After resampling:

In [46]:
count_sessions_of_class_1(resampled_df)

100%|██████████| 81252/81252 [01:07<00:00, 1195.58it/s]


count    74969.000000
mean         0.475116
std          0.499384
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
dtype: float64